In [1]:
import pandas as pd

## Settings and Daily Locomoter Activity

In [2]:
"""
Pseudo-code
Read in the monitor file and give the following column names when read in file or create dataframe:
Index,Date,Time,LD-DD,Status,Extras,Monitor_Number,Tube_Number,Data_Type,Unused,Light_Sensor,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,data_11,data_12,data_13,data_14,data_15,data_16,data_17,data_18,data_19,data_20,data_21,data_22,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32

Set variables - LD_start = 06:00, counts_per_day_alive = 100, LD_DD_anlysis = 'LD', start_date = 2/24/2024, end_date = 2/27/2024
Create column called LD_DD with LD_start + 12:00 = LD else DD
Filter on date between start_date & end_date
Filter on LD_DD based on LD_DD_analysis
Create column called date_LD_DD based on concat of date + LD_DD
Group by date_LD_DD and sum each column of data_1 - data_32
Loop through data_1 .. data_32 and determine the alive flies & dead flies - any fly with less than counts_per_day_alive is removed from the analysis - remove from the dataframe and keep number_of_dead_flies
Calculate number of alive flies, number of dead flies, Mean, SD, SEM - stats only on alive flies
Make sure the output is the same as the shiny app
"""

"\nPseudo-code\nRead in the monitor file and give the following column names when read in file or create dataframe:\nIndex,Date,Time,LD-DD,Status,Extras,Monitor_Number,Tube_Number,Data_Type,Unused,Light_Sensor,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,data_11,data_12,data_13,data_14,data_15,data_16,data_17,data_18,data_19,data_20,data_21,data_22,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32\n\nSet variables - LD_start = 06:00, counts_per_day_alive = 100, LD_DD_anlysis = 'LD', start_date = 2/24/2024, end_date = 2/27/2024\nCreate column called LD_DD with LD_start + 12:00 = LD else DD\nFilter on date between start_date & end_date\nFilter on LD_DD based on LD_DD_analysis\nCreate column called date_LD_DD based on concat of date + LD_DD\nGroup by date_LD_DD and sum each column of data_1 - data_32\nLoop through data_1 .. data_32 and determine the alive flies & dead flies - any fly with less than counts_per_day_alive is removed fro

### Set variables - LD_start = 06:00, counts_per_day_alive = 100, LD_DD_anlysis = 'LD', start_date = 2/24/2024, end_date = 2/27/2024


In [3]:
#Set variables - LD_start = 06:00, counts_per_day_alive = 100, LD_DD_anlysis = 'LD', start_date = 2/24/2024, end_date = 2/27/2024
LD_start = '06:00'
counts_per_day_alive = 100
LD_DD_analysis = 'LD'
start_date = '24 Feb 24'
end_date = '27 Feb 24'
condition = 'wt'

### Read in the monitor file and give the following column names when read in file or create dataframe:
### Index,Date,Time,LD-DD,Status,Extras,Monitor_Number,Tube_Number,Data_Type,Unused,Light_Sensor,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,data_11,data_12,data_13,data_14,data_15,data_16,data_17,data_18,data_19,data_20,data_21,data_22,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32

In [4]:
# Read in monitor9.txt, path is /home/twall5/sleep_analysis-3/examples/Monitor9.txt
df = pd.read_csv('../examples/Monitor9.txt', sep='\t', header=None)

# Give the columns names
# Index,Date,Time,LD-DD,Status,Extras,Monitor_Number,Tube_Number,Data_Type,Unused,Light_Sensor,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,data_11,data_12,data_13,data_14,data_15,data_16,data_17,data_18,data_19,data_20,data_21,data_22,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
df.columns = ['Index','Date','Time','LD-DD','Status','Monitor_Number','Extras','Tube_Number','Data_Type','Light_Sensor','data_1','data_2','data_3','data_4','data_5','data_6','data_7','data_8','data_9','data_10','data_11','data_12','data_13','data_14','data_15','data_16','data_17','data_18','data_19','data_20','data_21','data_22','data_23','data_24','data_25','data_26','data_27','data_28','data_29','data_30','data_31','data_32']

df['Date_Time'] = df['Date'] + ' ' + df['Time']
df['Date_Time'] = pd.to_datetime(df['Date_Time'], format='%d %b %y %H:%M:%S')

# change Date to date columns with format yyyy-mm-dd
df['Date'] = pd.to_datetime(df['Date'], format='%d %b %y')
df.head(2)

,Index,Date,Time,LD-DD,Status,Monitor_Number,Extras,Tube_Number,Data_Type,Light_Sensor,...,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32,Date_Time
0,2,2024-02-23,11:03:00,1,0,9,0,MT,0,1,...,11,22,8,20,19,29,64,14,37,2024-02-23 11:03:00
1,3,2024-02-23,11:04:00,1,0,9,0,MT,0,1,...,6,25,2,37,23,38,37,10,31,2024-02-23 11:04:00


### Create column called LD_DD with LD_start + 12:00 = LD else DD

In [5]:
#get the time in the date_time column, if the time is from 06:00 to 18:00 then it is LD, else it is DD
df['LD_DD'] = df['Date_Time'].dt.time
df['LD_DD'] = df['LD_DD'].apply(lambda x: 'LD' if x >= pd.to_datetime(LD_start).time() and x <= pd.to_datetime('18:00').time() else 'DD')
print(len(df))
df.head(2)

7352


,Index,Date,Time,LD-DD,Status,Monitor_Number,Extras,Tube_Number,Data_Type,Light_Sensor,...,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32,Date_Time,LD_DD
0,2,2024-02-23,11:03:00,1,0,9,0,MT,0,1,...,22,8,20,19,29,64,14,37,2024-02-23 11:03:00,LD
1,3,2024-02-23,11:04:00,1,0,9,0,MT,0,1,...,25,2,37,23,38,37,10,31,2024-02-23 11:04:00,LD


### Filter on date between start_date & end_date

In [6]:
#Make start_date start at 00:00:00 and end_date end at 23:59:59
start_date = start_date + ' 00:00:00'
end_date = end_date + ' 23:59:59'

#Filter on date between start_date & end_date, make sure you include the entire day of end_date
df = df[(df['Date_Time'] >= pd.to_datetime(start_date)) & (df['Date_Time'] <= pd.to_datetime(end_date))]
print(len(df))
df.head(2)

5760


,Index,Date,Time,LD-DD,Status,Monitor_Number,Extras,Tube_Number,Data_Type,Light_Sensor,...,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32,Date_Time,LD_DD
777,779,2024-02-24,00:00:00,1,0,9,0,MT,0,0,...,0,0,0,0,0,0,0,0,2024-02-24 00:00:00,DD
778,780,2024-02-24,00:01:00,1,0,9,0,MT,0,0,...,0,0,0,0,5,0,0,0,2024-02-24 00:01:00,DD


### group by date and sum the counts

In [7]:
# define the numeric columns with data_1 - data_32
numeric_cols = [f'data_{i}' for i in range(1, 33)]

In [8]:
# groupby date and sum each column of data_1 - data_32
df_grouped = df.groupby(['Date'])[numeric_cols].sum()
df_grouped.head()

,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,...,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-24,0,0,12936,3830,9277,6754,8116,8306,11292,8429,...,15169,12450,4519,3783,12336,13946,8444,14507,4976,8440
2024-02-25,0,0,13459,4505,17518,8717,9796,5370,11416,7439,...,12542,12650,7066,412,9434,14060,6970,13897,3912,9254
2024-02-26,0,0,14480,4974,14200,13696,12093,7789,11087,7415,...,12041,20878,12378,146,9103,15338,9793,15242,4608,9115
2024-02-27,0,0,22332,4814,11883,18053,12522,10084,11225,8750,...,12546,26041,13896,31,10464,17539,13359,16658,5638,12125


### calculate the number of dead and alive flies based on counts_per_day_alive

In [9]:
# loop through each date and determine the number of alive flies and dead flies
dead_flies = {}
for date in df_grouped.index:
    for col in numeric_cols:
        if df_grouped.loc[date, col] < counts_per_day_alive:
            # check if date is in dead_flies, if not add it
            if col not in dead_flies:
                dead_flies[col] = 1
            else:
                dead_flies[col] += 1
            #df_grouped.loc[date, col] = 0
print(dead_flies)

{'data_1': 4, 'data_2': 4, 'data_26': 1}


### calculate mean of each column from data_1 to data_32

In [10]:
# create a row for mean
df_grouped.loc['mean', numeric_cols] = df_grouped[numeric_cols].mean()
df_grouped.head()

,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,...,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-24 00:00:00,0.0,0.0,12936.00,3830.00,9277.0,6754.0,8116.00,8306.00,11292.0,8429.00,...,15169.0,12450.00,4519.00,3783.0,12336.00,13946.00,8444.0,14507.0,4976.0,8440.0
2024-02-25 00:00:00,0.0,0.0,13459.00,4505.00,17518.0,8717.0,9796.00,5370.00,11416.0,7439.00,...,12542.0,12650.00,7066.00,412.0,9434.00,14060.00,6970.0,13897.0,3912.0,9254.0
2024-02-26 00:00:00,0.0,0.0,14480.00,4974.00,14200.0,13696.0,12093.00,7789.00,11087.0,7415.00,...,12041.0,20878.00,12378.00,146.0,9103.00,15338.00,9793.0,15242.0,4608.0,9115.0
2024-02-27 00:00:00,0.0,0.0,22332.00,4814.00,11883.0,18053.0,12522.00,10084.00,11225.0,8750.00,...,12546.0,26041.00,13896.00,31.0,10464.00,17539.00,13359.0,16658.0,5638.0,12125.0
mean,0.0,0.0,15801.75,4530.75,13219.5,11805.0,10631.75,7887.25,11255.0,8008.25,...,13074.5,18004.75,9464.75,1093.0,10334.25,15220.75,9641.5,15076.0,4783.5,9733.5


In [11]:
available_channels = numeric_cols - dead_flies.keys()
#list(available_channels)

In [12]:
n_of_alive_flies = len(numeric_cols - dead_flies.keys())
n_of_dead_flies = len(dead_flies.keys())
n_of_all_flies = n_of_alive_flies + n_of_dead_flies
print(f"Number of alive flies: {n_of_alive_flies}")
print(f"Number of dead flies: {n_of_dead_flies}")
print(f"Number of all flies: {n_of_all_flies}")

Number of alive flies: 29
Number of dead flies: 3
Number of all flies: 32


In [13]:
# On the mean row, add up all the columns from ch1 to ch32, put this into a new variable called mean
mean = df_grouped.loc['mean', list(available_channels)].sum() / n_of_alive_flies

# Find the standard deviation from the available_channels mean
std = df_grouped.loc['mean', list(available_channels)].std()

# Find the Standard Error of the Mean (SEM) from the available_channels mean
sem = std / (n_of_alive_flies ** 0.5)

#Create a new dataframe called Daily_locomotor_activity with the columns Condition, Light_Cycle, Mean, SD, SEM, N_of_alive_flies, N_of_dead_flies, N_of_all_flies
daily_locomotor_activity = pd.DataFrame(data={'Condition': [condition], 'Light_Cycle': [LD_DD_analysis], 'Mean': [mean], 'SD': [std], 'SEM': [sem], 'N_of_alive_flies': [n_of_alive_flies], 'N_of_dead_flies': [n_of_dead_flies], 'N_of_all_flies': [n_of_all_flies]})
daily_locomotor_activity

,Condition,Light_Cycle,Mean,SD,SEM,N_of_alive_flies,N_of_dead_flies,N_of_all_flies
0,wt,LD,10190.112069,4148.973345,770.445008,29,3,32


### Calculate Individual daily locomotor activity data.csv

In [14]:
df_grouped_reindex = df_grouped.reset_index()

# drop the mean row
df_grouped_reindex = df_grouped_reindex[df_grouped_reindex['Date'] != 'mean']
df_grouped_reindex.head()

,Date,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,...,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
0,2024-02-24 00:00:00,0.0,0.0,12936.0,3830.0,9277.0,6754.0,8116.0,8306.0,11292.0,...,15169.0,12450.0,4519.0,3783.0,12336.0,13946.0,8444.0,14507.0,4976.0,8440.0
1,2024-02-25 00:00:00,0.0,0.0,13459.0,4505.0,17518.0,8717.0,9796.0,5370.0,11416.0,...,12542.0,12650.0,7066.0,412.0,9434.0,14060.0,6970.0,13897.0,3912.0,9254.0
2,2024-02-26 00:00:00,0.0,0.0,14480.0,4974.0,14200.0,13696.0,12093.0,7789.0,11087.0,...,12041.0,20878.0,12378.0,146.0,9103.0,15338.0,9793.0,15242.0,4608.0,9115.0
3,2024-02-27 00:00:00,0.0,0.0,22332.0,4814.0,11883.0,18053.0,12522.0,10084.0,11225.0,...,12546.0,26041.0,13896.0,31.0,10464.0,17539.0,13359.0,16658.0,5638.0,12125.0


In [15]:
# unpivot the dataframe with Date as the index and the columns as the data_1 - data_32
df_unpivoted = df_grouped_reindex.melt(id_vars=['Date'], value_vars=numeric_cols, var_name='Channel', value_name='Counts')
df_unpivoted.to_csv('../examples/individual_daily_locomotor_activity_data.csv', index=False)
df_unpivoted.head(10)

,Date,Channel,Counts
0,2024-02-24,data_1,0.0
1,2024-02-25,data_1,0.0
2,2024-02-26,data_1,0.0
3,2024-02-27,data_1,0.0
4,2024-02-24,data_2,0.0
5,2024-02-25,data_2,0.0
6,2024-02-26,data_2,0.0
7,2024-02-27,data_2,0.0
8,2024-02-24,data_3,12936.0
9,2024-02-25,data_3,13459.0


### Calculate Average_individual_daily_locomotor_activity.csv

In [16]:
# create a new df with only the alive flies
df_unpivoted_alive = df_unpivoted[df_unpivoted['Channel'].isin(available_channels)]
df_unpivoted_alive.head()

,Date,Channel,Counts
8,2024-02-24,data_3,12936.0
9,2024-02-25,data_3,13459.0
10,2024-02-26,data_3,14480.0
11,2024-02-27,data_3,22332.0
12,2024-02-24,data_4,3830.0


In [17]:
avg_ind_df = df_unpivoted_alive.groupby('Channel')['Counts'].mean().reset_index()
avg_ind_df.to_csv('../examples/average_daily_locomotor_activity_data.csv', index=False)
avg_ind_df.head()

,Channel,Counts
0,data_10,8008.25
1,data_11,8733.00
2,data_12,6015.25
3,data_13,6090.25
4,data_14,6773.00


### calculate locomotor_activity_by_day.csv

In [18]:
# calculate the mean, sum, sd, sem for each day
df_by_day = df_unpivoted_alive.groupby('Date')['Counts'].agg(['mean', 'sum', 'std', 'sem']).reset_index()
df_by_day.to_csv('../examples/daily_locomotor_activity_data.csv', index=False)
df_by_day.head()

,Date,mean,sum,std,sem
0,2024-02-24,9306.103448,269877.0,3638.126876,675.583201
1,2024-02-25,9213.068966,267179.0,4069.985196,755.777277
2,2024-02-26,10444.758621,302898.0,4769.514416,885.676592
3,2024-02-27,11796.517241,342099.0,5903.262637,1096.208352


In [19]:
loc_by_day_df = df_unpivoted_alive.groupby('Date')['Counts']
loc_by_day_df = df_unpivoted_alive.groupby('Date')['Counts'].mean().reset_index()
loc_by_day_df.to_csv('../examples/locomotor_activity_by_day.csv', index=False)
loc_by_day_df.head()

,Date,Counts
0,2024-02-24,9306.103448
1,2024-02-25,9213.068966
2,2024-02-26,10444.758621
3,2024-02-27,11796.517241


### calculate daytime_nighttime_activity_in_LD.csv

In [20]:
df_ld_dd = df.groupby(['Date','LD_DD'])[numeric_cols].sum()
df_ld_dd.reset_index(inplace=True)
df_ld_dd.head()


,Date,LD_DD,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,...,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
0,2024-02-24,DD,0,0,3191,523,2912,272,1508,1125,...,829,791,1105,552,2313,1663,3780,2451,541,837
1,2024-02-24,LD,0,0,9745,3307,6365,6482,6608,7181,...,14340,11659,3414,3231,10023,12283,4664,12056,4435,7603
2,2024-02-25,DD,0,0,2649,971,4957,321,1960,406,...,917,1126,1120,48,1397,1919,1153,2120,605,806
3,2024-02-25,LD,0,0,10810,3534,12561,8396,7836,4964,...,11625,11524,5946,364,8037,12141,5817,11777,3307,8448
4,2024-02-26,DD,0,0,3275,843,5255,940,2813,790,...,1309,4312,1569,4,2209,2459,2276,2087,993,2014


In [21]:
df_melt_ld_dd = df_ld_dd.melt(id_vars=['Date','LD_DD'], value_vars=numeric_cols, var_name='Channel', value_name='Counts')
df_melt_ld_dd.head()

,Date,LD_DD,Channel,Counts
0,2024-02-24,DD,data_1,0
1,2024-02-24,LD,data_1,0
2,2024-02-25,DD,data_1,0
3,2024-02-25,LD,data_1,0
4,2024-02-26,DD,data_1,0


In [22]:
df_melt_ld_dd_alive = df_melt_ld_dd[df_melt_ld_dd['Channel'].isin(available_channels)]
df_melt_ld_dd_alive.head()

,Date,LD_DD,Channel,Counts
16,2024-02-24,DD,data_3,3191
17,2024-02-24,LD,data_3,9745
18,2024-02-25,DD,data_3,2649
19,2024-02-25,LD,data_3,10810
20,2024-02-26,DD,data_3,3275


In [23]:
df_melt_ld_dd_alive.tail()

,Date,LD_DD,Channel,Counts
251,2024-02-25,LD,data_32,8448
252,2024-02-26,DD,data_32,2014
253,2024-02-26,LD,data_32,7101
254,2024-02-27,DD,data_32,2787
255,2024-02-27,LD,data_32,9338


In [24]:
# calculate the mean, sum, sd, sem for each LD & DD in the LD_DD column
df_by_LD_DD = df_melt_ld_dd_alive.groupby('LD_DD')['Counts'].agg(['mean', 'sum', 'std', 'sem']).reset_index()
df_by_LD_DD.to_csv('../examples/daytime_nighttime_activity_in_LD.csv', index=False)
df_by_LD_DD.head()

,LD_DD,mean,sum,std,sem
0,DD,1623.362069,188310,1307.786011,121.424883
1,LD,8566.750000,993743,3809.681988,353.720093


## Sleep Analysis

In [25]:
df.head(2)

,Index,Date,Time,LD-DD,Status,Monitor_Number,Extras,Tube_Number,Data_Type,Light_Sensor,...,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32,Date_Time,LD_DD
777,779,2024-02-24,00:00:00,1,0,9,0,MT,0,0,...,0,0,0,0,0,0,0,0,2024-02-24 00:00:00,DD
778,780,2024-02-24,00:01:00,1,0,9,0,MT,0,0,...,0,0,0,0,5,0,0,0,2024-02-24 00:01:00,DD


In [26]:
# filter on 2024-02-24 00:00:00
df_24 = df[df['Date_Time'] == '2024-02-24 00:00:00']
df_24

,Index,Date,Time,LD-DD,Status,Monitor_Number,Extras,Tube_Number,Data_Type,Light_Sensor,...,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32,Date_Time,LD_DD
777,779,2024-02-24,00:00:00,1,0,9,0,MT,0,0,...,0,0,0,0,0,0,0,0,2024-02-24,DD


In [27]:
cols = ['Date','Time','Date_Time','LD_DD'] + numeric_cols
df_24 = df_24[cols]
df_24

,Date,Time,Date_Time,LD_DD,data_1,data_2,data_3,data_4,data_5,data_6,...,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
777,2024-02-24,00:00:00,2024-02-24,DD,0,0,0,0,5,0,...,0,4,0,0,0,0,0,0,0,0


In [28]:
# add a new column called mean with the mean of columns data_1 - data_32
df_24['mean'] = df_24[numeric_cols].mean(axis=1)
df_24

/tmp/ipykernel_1504948/2897340146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_24['mean'] = df_24[numeric_cols].mean(axis=1)


,Date,Time,Date_Time,LD_DD,data_1,data_2,data_3,data_4,data_5,data_6,...,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32,mean
777,2024-02-24,00:00:00,2024-02-24,DD,0,0,0,0,5,0,...,4,0,0,0,0,0,0,0,0,0.8125


In [29]:
# df_24_wake
df_24_wake = df_24.copy()

# convert all values above 1 to 1 in the numeric_columns
df_24_wake[numeric_cols] = df_24_wake[numeric_cols].applymap(lambda x: 1 if x > 1 else x)

df_24_wake['mean'] = df_24_wake[numeric_cols].mean(axis=1)

df_24_wake

/tmp/ipykernel_1504948/3201691224.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_24_wake[numeric_cols] = df_24_wake[numeric_cols].applymap(lambda x: 1 if x > 1 else x)


,Date,Time,Date_Time,LD_DD,data_1,data_2,data_3,data_4,data_5,data_6,...,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32,mean
777,2024-02-24,00:00:00,2024-02-24,DD,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0.125


In [30]:
"""
Sleep events are identified using a sliding window algorithm of 5 min width and 1 min sliding interval. 
A sleep event is detected if all 5 activity readings in a window equal 0. 
Sleep events are then averaged over all individual flies in a condition. 
Sleep = 1 means that all flies were inactive in a 5 min window, and sleep = 0 means that all flies scored at least 1 count in a 5 min window. 
The resolution of sleep profiles is further binned over an average. 
The binning window is 30
"""

'\nSleep events are identified using a sliding window algorithm of 5 min width and 1 min sliding interval. \nA sleep event is detected if all 5 activity readings in a window equal 0. \nSleep events are then averaged over all individual flies in a condition. \nSleep = 1 means that all flies were inactive in a 5 min window, and sleep = 0 means that all flies scored at least 1 count in a 5 min window. \nThe resolution of sleep profiles is further binned over an average. \nThe binning window is 30\n'

In [31]:
# based on the previous statement, we need to create a new dataframe with the sleep events

# create a new dataframe with the sleep events
df_sleep = df.copy()
df_sleep.head()

,Index,Date,Time,LD-DD,Status,Monitor_Number,Extras,Tube_Number,Data_Type,Light_Sensor,...,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32,Date_Time,LD_DD
777,779,2024-02-24,00:00:00,1,0,9,0,MT,0,0,...,0,0,0,0,0,0,0,0,2024-02-24 00:00:00,DD
778,780,2024-02-24,00:01:00,1,0,9,0,MT,0,0,...,0,0,0,0,5,0,0,0,2024-02-24 00:01:00,DD
779,781,2024-02-24,00:02:00,1,0,9,0,MT,0,0,...,1,0,0,0,6,0,0,0,2024-02-24 00:02:00,DD
780,782,2024-02-24,00:03:00,1,0,9,0,MT,0,0,...,0,0,0,0,11,2,0,0,2024-02-24 00:03:00,DD
781,783,2024-02-24,00:04:00,1,0,9,0,MT,0,0,...,0,0,0,0,7,0,0,0,2024-02-24 00:04:00,DD


In [32]:
# create a function for Zeitgeber time where the start of the light cycle at 06:00 is equal to 00:00 and 05:59 is equal to 23:59
def zeitgeber_time(dt):
    if dt.time() >= pd.to_datetime('06:00').time() and dt.time() <= pd.to_datetime('23:59').time():
        # subtract 6 hours from the time
        six_hours = timedelta(hours=6)
        zt_time = dt - six_hours
    else:
        # add 18 hours to the time
        eighteen_hours = timedelta(hours=18)
        zt_time = dt + eighteen_hours
    return zt_time

In [33]:
# create datetime object with 2024-02-24 05:59:00 using the datetime module
from datetime import datetime, timedelta
test_time = datetime(2024, 2, 24, 5, 59, 0)
print(test_time)

test_time_zt = zeitgeber_time(test_time)
print(test_time_zt)

2024-02-24 05:59:00
2024-02-24 23:59:00


In [34]:
# add the zt_time column to the dataframe
df_sleep['zt_time'] = df_sleep['Date_Time'].apply(zeitgeber_time)

In [35]:
df_sleep.head()

,Index,Date,Time,LD-DD,Status,Monitor_Number,Extras,Tube_Number,Data_Type,Light_Sensor,...,data_26,data_27,data_28,data_29,data_30,data_31,data_32,Date_Time,LD_DD,zt_time
777,779,2024-02-24,00:00:00,1,0,9,0,MT,0,0,...,0,0,0,0,0,0,0,2024-02-24 00:00:00,DD,2024-02-24 18:00:00
778,780,2024-02-24,00:01:00,1,0,9,0,MT,0,0,...,0,0,0,5,0,0,0,2024-02-24 00:01:00,DD,2024-02-24 18:01:00
779,781,2024-02-24,00:02:00,1,0,9,0,MT,0,0,...,0,0,0,6,0,0,0,2024-02-24 00:02:00,DD,2024-02-24 18:02:00
780,782,2024-02-24,00:03:00,1,0,9,0,MT,0,0,...,0,0,0,11,2,0,0,2024-02-24 00:03:00,DD,2024-02-24 18:03:00
781,783,2024-02-24,00:04:00,1,0,9,0,MT,0,0,...,0,0,0,7,0,0,0,2024-02-24 00:04:00,DD,2024-02-24 18:04:00


In [36]:
numeric_cols

['data_1',
 'data_2',
 'data_3',
 'data_4',
 'data_5',
 'data_6',
 'data_7',
 'data_8',
 'data_9',
 'data_10',
 'data_11',
 'data_12',
 'data_13',
 'data_14',
 'data_15',
 'data_16',
 'data_17',
 'data_18',
 'data_19',
 'data_20',
 'data_21',
 'data_22',
 'data_23',
 'data_24',
 'data_25',
 'data_26',
 'data_27',
 'data_28',
 'data_29',
 'data_30',
 'data_31',
 'data_32']

In [37]:
# trim df_sleep to Date_Time, zt_time, data_1 - data_32
df_sleep_trim = df_sleep[['Date_Time', 'zt_time'] + numeric_cols]
df_sleep_trim.head()

,Date_Time,zt_time,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,...,data_23,data_24,data_25,data_26,data_27,data_28,data_29,data_30,data_31,data_32
777,2024-02-24 00:00:00,2024-02-24 18:00:00,0,0,0,0,5,0,0,0,...,0,4,0,0,0,0,0,0,0,0
778,2024-02-24 00:01:00,2024-02-24 18:01:00,0,0,0,0,0,1,0,0,...,0,4,0,0,0,0,5,0,0,0
779,2024-02-24 00:02:00,2024-02-24 18:02:00,0,0,0,0,0,2,0,0,...,0,10,1,0,0,0,6,0,0,0
780,2024-02-24 00:03:00,2024-02-24 18:03:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,11,2,0,0
781,2024-02-24 00:04:00,2024-02-24 18:04:00,0,0,0,0,9,0,0,0,...,0,3,0,0,0,0,7,0,0,0


In [38]:
# remove the dead flies from the dataframe
df_sleep_trim_alive = df_sleep_trim.drop(dead_flies.keys(), axis=1)
df_sleep_trim_alive.head()

,Date_Time,zt_time,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,...,data_22,data_23,data_24,data_25,data_27,data_28,data_29,data_30,data_31,data_32
777,2024-02-24 00:00:00,2024-02-24 18:00:00,0,0,5,0,0,0,0,0,...,6,0,4,0,0,0,0,0,0,0
778,2024-02-24 00:01:00,2024-02-24 18:01:00,0,0,0,1,0,0,0,0,...,13,0,4,0,0,0,5,0,0,0
779,2024-02-24 00:02:00,2024-02-24 18:02:00,0,0,0,2,0,0,0,0,...,5,0,10,1,0,0,6,0,0,0
780,2024-02-24 00:03:00,2024-02-24 18:03:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,11,2,0,0
781,2024-02-24 00:04:00,2024-02-24 18:04:00,0,0,9,0,0,0,0,0,...,1,0,3,0,0,0,7,0,0,0


In [39]:
numeric_cols

['data_1',
 'data_2',
 'data_3',
 'data_4',
 'data_5',
 'data_6',
 'data_7',
 'data_8',
 'data_9',
 'data_10',
 'data_11',
 'data_12',
 'data_13',
 'data_14',
 'data_15',
 'data_16',
 'data_17',
 'data_18',
 'data_19',
 'data_20',
 'data_21',
 'data_22',
 'data_23',
 'data_24',
 'data_25',
 'data_26',
 'data_27',
 'data_28',
 'data_29',
 'data_30',
 'data_31',
 'data_32']

In [40]:
alive_cols = [col for col in numeric_cols if col not in dead_flies.keys()]
alive_cols

['data_3',
 'data_4',
 'data_5',
 'data_6',
 'data_7',
 'data_8',
 'data_9',
 'data_10',
 'data_11',
 'data_12',
 'data_13',
 'data_14',
 'data_15',
 'data_16',
 'data_17',
 'data_18',
 'data_19',
 'data_20',
 'data_21',
 'data_22',
 'data_23',
 'data_24',
 'data_25',
 'data_27',
 'data_28',
 'data_29',
 'data_30',
 'data_31',
 'data_32']

In [41]:
df_sleep_calc_list = []

# loop through df_sleep_trim_alive
for i in range(0, len(df_sleep_trim_alive)):
#for i in range(0,2):
    current_row_list = [df_sleep_trim_alive.iloc[i]['Date_Time'], df_sleep_trim_alive.iloc[i]['zt_time']]
    for col in alive_cols:
        # get the 5 min window
        five_min_window = df_sleep_trim_alive.iloc[i:i+5][col]
        #print(five_min_window)
        five_min_window_sum = five_min_window.sum()
        #print(five_min_window_sum)

        #print(f"index={i}, Date_Time={df_sleep_trim_alive.iloc[i]['Date_Time']}, zt_time={df_sleep_trim_alive.iloc[i]['zt_time']}, col={col}, five_min_window={five_min_window}")
        if five_min_window_sum == 0:
            sleep = 1
        else:
            sleep = 0
        # add the sleep event to the df_sleep_calc_list
        current_row_list.append(sleep)
    df_sleep_calc_list.append(current_row_list)

# df_sleep_calc_list

In [42]:
# create a new dataframe with the sleep events
df_sleep_calc = pd.DataFrame(df_sleep_calc_list, columns=['Date_Time', 'zt_time'] + alive_cols)
df_sleep_calc.head()

,Date_Time,zt_time,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,...,data_22,data_23,data_24,data_25,data_27,data_28,data_29,data_30,data_31,data_32
0,2024-02-24 00:00:00,2024-02-24 18:00:00,1,1,0,0,1,1,1,1,...,0,1,0,0,1,1,0,0,1,1
1,2024-02-24 00:01:00,2024-02-24 18:01:00,1,1,0,0,1,1,1,1,...,0,1,0,0,1,1,0,0,1,1
2,2024-02-24 00:02:00,2024-02-24 18:02:00,1,1,0,0,1,1,1,1,...,0,1,0,0,1,1,0,0,1,1
3,2024-02-24 00:03:00,2024-02-24 18:03:00,1,0,0,0,1,0,1,1,...,0,1,0,0,0,1,0,0,1,1
4,2024-02-24 00:04:00,2024-02-24 18:04:00,1,0,0,0,1,0,1,1,...,0,1,0,0,0,1,0,0,1,1


In [43]:
# create two new columns with mean and sem of the data_1 - data_32 columns
df_sleep_calc['mean'] = df_sleep_calc[alive_cols].mean(axis=1)
df_sleep_calc['sem'] = df_sleep_calc[alive_cols].sem(axis=1)
df_sleep_calc.head()

,Date_Time,zt_time,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,...,data_24,data_25,data_27,data_28,data_29,data_30,data_31,data_32,mean,sem
0,2024-02-24 00:00:00,2024-02-24 18:00:00,1,1,0,0,1,1,1,1,...,0,0,1,1,0,0,1,1,0.724138,0.084465
1,2024-02-24 00:01:00,2024-02-24 18:01:00,1,1,0,0,1,1,1,1,...,0,0,1,1,0,0,1,1,0.689655,0.087430
2,2024-02-24 00:02:00,2024-02-24 18:02:00,1,1,0,0,1,1,1,1,...,0,0,1,1,0,0,1,1,0.689655,0.087430
3,2024-02-24 00:03:00,2024-02-24 18:03:00,1,0,0,0,1,0,1,1,...,0,0,0,1,0,0,1,1,0.551724,0.093984
4,2024-02-24 00:04:00,2024-02-24 18:04:00,1,0,0,0,1,0,1,1,...,0,0,0,1,0,0,1,1,0.551724,0.093984


In [44]:
df_sleep_calc.tail()

,Date_Time,zt_time,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,...,data_24,data_25,data_27,data_28,data_29,data_30,data_31,data_32,mean,sem
5755,2024-02-27 23:55:00,2024-02-27 17:55:00,0,1,1,0,1,1,1,1,...,1,0,1,1,0,1,1,1,0.724138,0.084465
5756,2024-02-27 23:56:00,2024-02-27 17:56:00,0,1,1,1,1,1,1,1,...,1,0,1,1,0,1,1,1,0.793103,0.076553
5757,2024-02-27 23:57:00,2024-02-27 17:57:00,0,1,1,1,1,1,1,1,...,1,0,1,1,0,1,1,1,0.793103,0.076553
5758,2024-02-27 23:58:00,2024-02-27 17:58:00,0,1,1,1,1,1,1,1,...,1,0,1,1,0,1,1,1,0.793103,0.076553
5759,2024-02-27 23:59:00,2024-02-27 17:59:00,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0.896552,0.057553


In [45]:
# output to csv
df_sleep_calc.to_csv('../examples/shiny_tool/sleep_analysis_results/daily_sleep_profiles_python.csv', index=False)

### bin the results to generate Average_sleep_profiles_in_LD.csv

In [46]:
# create column with just the time
avg_sleep_df = df_sleep_calc.copy()
avg_sleep_df['time'] = avg_sleep_df['Date_Time'].dt.time

# remove all columns except for the time and mean
avg_sleep_df = avg_sleep_df[['time', 'mean']]

# sort by time
avg_sleep_df.sort_values('time', inplace=True)

# reset the index
avg_sleep_df.reset_index(drop=True, inplace=True)

avg_sleep_df.head()

,time,mean
0,00:00:00,0.724138
1,00:00:00,0.793103
2,00:00:00,0.689655
3,00:00:00,0.758621
4,00:01:00,0.689655


In [47]:
avg_sleep_df.tail()

,time,mean
5755,23:58:00,0.724138
5756,23:59:00,0.793103
5757,23:59:00,0.724138
5758,23:59:00,0.689655
5759,23:59:00,0.896552


In [48]:
# set the bin to 30 minutes
bin = 30
avg_sleep_list = []
for i in range(0, len(avg_sleep_df), bin):
    #print(f"start={i}, end={i+bin-1}")
    start_time = pd.to_datetime(i, unit='m').time()
    end_time = pd.to_datetime(i+bin-1, unit='m').time()
    #print(f"start_time={start_time}, end_time={end_time}")
    
    # put the mean values into a list where the time is between start_time and end_time
    mean_values = avg_sleep_df[(avg_sleep_df['time'] >= start_time) & (avg_sleep_df['time'] <= end_time)]['mean'].tolist()
    
    # calculate the mean of the mean_values
    bin_mean = sum(mean_values) / len(mean_values)
    
    # calculate the sem of the mean_values
    bin_sem = pd.Series(mean_values).sem()
    
    avg_sleep_list.append([i, bin_mean, bin_sem])

avg_sleep_list_df = pd.DataFrame(avg_sleep_list, columns=['time', 'mean', 'sem'])
avg_sleep_list_df.head()

,time,mean,sem
0,0,0.735345,0.009787
1,30,0.740805,0.009015
2,60,0.773563,0.004719
3,90,0.749425,0.005348
4,120,0.755172,0.005358


In [49]:
# add column with zt_time
avg_sleep_list_df['zt_time'] = pd.to_datetime(avg_sleep_list_df['time'], unit='m').dt.time
avg_sleep_list_df.head()

,time,mean,sem,zt_time
0,0,0.735345,0.009787,00:00:00
1,30,0.740805,0.009015,00:30:00
2,60,0.773563,0.004719,01:00:00
3,90,0.749425,0.005348,01:30:00
4,120,0.755172,0.005358,02:00:00


### calculate Individual_day_night_sleep.csv

In [50]:
df_sleep_calc.head(2)

,Date_Time,zt_time,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,...,data_24,data_25,data_27,data_28,data_29,data_30,data_31,data_32,mean,sem
0,2024-02-24 00:00:00,2024-02-24 18:00:00,1,1,0,0,1,1,1,1,...,0,0,1,1,0,0,1,1,0.724138,0.084465
1,2024-02-24 00:01:00,2024-02-24 18:01:00,1,1,0,0,1,1,1,1,...,0,0,1,1,0,0,1,1,0.689655,0.087430


In [51]:
# create a function to set either 'Day' or 'Night' based on the Date_Time time if it is between 06:00 and 18:00 then it is 'Day' else it is 'Night'
def day_night(dt):
    if dt.time() >= pd.to_datetime('06:00').time() and dt.time() <= pd.to_datetime('18:00').time():
        return 'Day'
    else:
        return 'Night'

In [52]:
ind_day_night = df_sleep_calc.copy()
ind_day_night['Day_Night'] = ind_day_night['Date_Time'].apply(day_night)
ind_day_night.head(2)

,Date_Time,zt_time,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,...,data_25,data_27,data_28,data_29,data_30,data_31,data_32,mean,sem,Day_Night
0,2024-02-24 00:00:00,2024-02-24 18:00:00,1,1,0,0,1,1,1,1,...,0,1,1,0,0,1,1,0.724138,0.084465,Night
1,2024-02-24 00:01:00,2024-02-24 18:01:00,1,1,0,0,1,1,1,1,...,0,1,1,0,0,1,1,0.689655,0.087430,Night


In [53]:
# select the columns Day_Night and alive_cols
ind_day_night = ind_day_night[['Day_Night'] + alive_cols]
ind_day_night.head(2)

,Day_Night,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,data_11,...,data_22,data_23,data_24,data_25,data_27,data_28,data_29,data_30,data_31,data_32
0,Night,1,1,0,0,1,1,1,1,1,...,0,1,0,0,1,1,0,0,1,1
1,Night,1,1,0,0,1,1,1,1,1,...,0,1,0,0,1,1,0,0,1,1


In [54]:
# unpivot the dataframe with Day_Night as the index and the columns as the data_1 - data_32
ind_day_night_unpivoted = ind_day_night.melt(id_vars=['Day_Night'], value_vars=alive_cols, var_name='Channel', value_name='Counts')
ind_day_night_unpivoted.head(5)

,Day_Night,Channel,Counts
0,Night,data_3,1
1,Night,data_3,1
2,Night,data_3,1
3,Night,data_3,1
4,Night,data_3,1


In [55]:
# calculate the mean for the Counts column for each Day_Night and Channel
ind_day_night_mean = ind_day_night_unpivoted.groupby(['Day_Night', 'Channel'])['Counts'].mean().reset_index()

# sort by Channel and Day_Night
ind_day_night_mean.sort_values(['Channel', 'Day_Night'], inplace=True)

# reset index
ind_day_night_mean.reset_index(drop=True, inplace=True)

ind_day_night_mean.head(5)

,Day_Night,Channel,Counts
0,Day,data_10,0.223994
1,Night,data_10,0.670028
2,Day,data_11,0.326976
3,Night,data_11,0.702712
4,Day,data_12,0.376560


### Calculate Individual_sleep_activity_bout_data.csv

In [56]:
df_sleep_calc.head(2)

,Date_Time,zt_time,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,...,data_24,data_25,data_27,data_28,data_29,data_30,data_31,data_32,mean,sem
0,2024-02-24 00:00:00,2024-02-24 18:00:00,1,1,0,0,1,1,1,1,...,0,0,1,1,0,0,1,1,0.724138,0.084465
1,2024-02-24 00:01:00,2024-02-24 18:01:00,1,1,0,0,1,1,1,1,...,0,0,1,1,0,0,1,1,0.689655,0.087430


In [57]:
ind_sleep_bout_list = []

for col in alive_cols:
    #for col in ['data_10']:
    print(col)
    
    # loop through df_sleep_calc and determine when it switches from 1 to 0 or 0 to 1

    # get the first value of the column
    current_time = df_sleep_calc.iloc[0]['Date_Time']
    current_value = df_sleep_calc.iloc[0][col]
    #print(f"start_value={current_value}")
    for i in range(1, len(df_sleep_calc)):
    #for i in range(1, 1000):
        if df_sleep_calc.iloc[i-1][col] != df_sleep_calc.iloc[i][col]:
            #sleep_switch_list.append(df_sleep_calc.iloc[i]['zt_time'])
            #print(f"sleep_switch_list={sleep_switch_list}")
            new_value = df_sleep_calc.iloc[i][col]
                        
            # calcluate the time difference between the current and previous time
            bout_time = (df_sleep_calc.iloc[i]['Date_Time'] - current_time)
            
            # convert timedelta to minutes
            bout_time = bout_time.total_seconds() / 60
            
            #print(f"new_value={new_value}, current_time={current_time}, bout_time={bout_time}")
            ind_sleep_bout_list.append([col, current_time, new_value, bout_time])
            
            current_value = df_sleep_calc.iloc[i][col]
            current_time = df_sleep_calc.iloc[i]['Date_Time']
            #print(f"current_value={current_value}")

data_3
data_4
data_5
data_6
data_7
data_8
data_9
data_10
data_11
data_12
data_13
data_14
data_15
data_16
data_17
data_18
data_19
data_20
data_21
data_22
data_23
data_24
data_25
data_27
data_28
data_29
data_30
data_31
data_32


In [58]:
ind_sleep_bout_df = pd.DataFrame(ind_sleep_bout_list, columns=['Channel', 'Time', 'Value', 'Bout_Length'])
ind_sleep_bout_df.head()

,Channel,Time,Value,Bout_Length
0,data_3,2024-02-24 00:00:00,0,49.0
1,data_3,2024-02-24 00:49:00,1,9.0
2,data_3,2024-02-24 00:58:00,0,19.0
3,data_3,2024-02-24 01:17:00,1,23.0
4,data_3,2024-02-24 01:40:00,0,34.0


In [59]:
### output to csv
ind_sleep_bout_df.to_csv('../examples/shiny_tool/sleep_analysis_results/individual_sleep_activity_bout_data_python.csv', index=False)